In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [49]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()
        
    # handle cookies
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "CONTINUE"]')))
        if cookie:
            cookie.click()
        else:
            pass
    except Exception as e:
        print("No cookies tab found and the error is {e}") 
    
    # # click and get the login info
    # try:
    #     log_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Sign In"]')))
    #     if log_card:
    #         driver.execute_script("arguments[0].scrollIntoView();", log_card)
    #         log_card.click()
    # except Exception as e:
    #     print("No login")
            
    
    #  # get the username tab
    # try:
    #     provided_u_name = "fourbrotherstrading@icloud.com"
    #     user_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,"LoginUserName")))   
    #     user_name.send_keys(provided_u_name)
    # except Exception as e:
    #     print(f"No username tab found and the error is {e}")

    # # get password tab
    # try:
    #     provided_pass = "Muhssan7865"
    #     password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "LoginPassword")))
    #     password.send_keys(provided_pass)
    # except Exception as e:
    #     print(f"No password tab found and the error is {e}")

    # # get the login tab to enter the username and password
    # try:
    #     # login_tab_css = "submit"
    #     login_tab = driver.find_element(By.XPATH,  ".//button[@type='submit']")
    #     driver.execute_script("arguments[0].scrollIntoView();", login_tab)
    #     login_tab.click()
    #     for i in range(2):
    #         driver.back()
    # except Exception as e:
    #     print(f"No login tab found and the error is {e}")
    
    
        # get only number
# =====================================================================================
    try:
        num_cars = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[@aria-controls="lots"]'))).text.strip().split("(")[1]
        if num_cars:
            cars = int(num_cars.split(")")[0])
            
        # cars_list = num_cars.split(" ")
            # cars =  int(cars_list[1]) # get only number
            print(f"Total {cars} number of cars found")
    except Exception as e:
        print(f"No number of cars found and error is {e}")

# =====================================================================================
    # get the car link and click on it to proceed to main car information page
# =====================================================================================
    try:
        link = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-grid-lot  col-sm-6 col-md-4 col-lg-4"]')))
        if link:
        # now we have to click on the link to get the main car info page
            driver.execute_script("arguments[0].scrollIntoView();", link)
            link.click()
        else:
            pass
    except Exception as e:
        print(f"No link found and the error is {e}")
    
    results = [] # to append the dict values later on
    for i in range(cars):# we will make use of number of pages here

    # ===================================================================================== 
        # get the name of the car
    # =====================================================================================
        details = {}
        
        try:
            main_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="lot lot-details col-sm-6"]'))) 
            if main_card:
                
                # title and desc
                try:
                    title_card= WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text.strip().split("\n")
                    if title_card:
                        details['Title'] = title_card[0]
                        details['Desc'] = title_card[1]
                    else:
                        details['Title'] = 'na'
                        details['Desc'] = "na"
                except Exception as e:
                    print("No car title")
                       
                # est price
                try:
                    est_price =WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="estimate"]'))).text.strip().split("\n") 
                    if est_price:
                        details[est_price[0]] = est_price[1]
                    else:
                        print("No car est price")
                except Exception as e:
                    print("No car est price")
                
                # other lot det
                try:
                    det_card = WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="lot-desc"])[1]/p[1]'))).text.strip().split("\n")
                    if det_card:
                        details['Car_det'] = det_card
                    else:
                        details['Car_det'] = "na"
                except Exception as e:
                    print(f"No details info and the error is {e}")
                
                # list information
                try:
                    list_info = WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="lot-desc"])[1]')))
                    if list_info:
                        list_ul = list_info.find_element(By.TAG_NAME, 'ul').text.strip().split("\n")
                        if list_ul:
                            details['List_info'] = list_ul
                        else:
                            details['List_info'] = "na"
                except Exception as e:
                    print("No list info")
                
                # other information
                try:
                    info_card = WebDriverWait(main_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@style="text-align: justify;"]'))).text.strip().split("\n")
                    if info_card:
                        details['Car_text_dec'] = info_card
                    else:
                        details['Car_text_dec'] = "n"
                except Exception as e:
                    print("No car text details")      
        except Exception as e:
            print(f"No car name found and the error is {e}")
            
        # imgs
        try:
            img_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="lot-gallery-wrapper-horizontal"]'))) 
            if img_card:
                # driver.execute_script("arguments[0].scrollIntoView();", img_card)
                imgs =WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'li')))  
                imgs_lst=[]
                if imgs:
                    for li in imgs:
                        img = li.find_element(By.TAG_NAME, 'img').get_attribute("src")
                    #  = [img.get_attribute("src") for img in imgs]
                        imgs_lst.append(img)
                    details['Images'] = ", ".join(imgs_lst)
        except Exception as e:
            print("No images found")   
            
        # date ande time
        try:
            date_time =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/h3[@class="date-title"]'))).text.strip()
            if date_time:
                details['Date/Time'] = date_time
            else:
                details['Date/Time'] = "na"
        except Exception as e:
            print("No date and time")
        
        # location
        try:
            loc= WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/h2[@class="auction-title"]'))).text.strip()
            if loc:
                details['Location'] = loc
            else:
                details['Location'] = "na"
        except Exception as e:
            print("No Location found")
        results.append(details)
    
         # =====================================================================================
        # get the next button
    # =====================================================================================
        try:
            next_button = driver.find_element(By.XPATH, './/a[@oldtitle="Next lot"]')
            if next_button:
                next_button.click()
            else:
                break
        except Exception as e:
            print("No next button found hence no more cars are available")
            
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("hnh.csv", index=False)
    
    
    
    time.sleep(4)
    driver.quit()
    
path = "https://www.handh.co.uk/auction/search/?au=548"
scrape(path)
    
    

Total 37 number of cars found
No car title
No car text details
No car text details
No car title
No car title
No car text details
No car text details
No car text details
No car text details
No car text details
No car est price
No car text details
No car est price
No car text details
No car est price
No car text details
No car title
No car text details
No car est price
No car text details
No car est price
No car text details
No car title
No car text details
No car title
No car title
No car text details
No car est price
No car text details
No car est price
No car title
No car text details
No car title
No car title
No car title
No car title
No car title
No car title
No car text details
No car title
No next button found hence no more cars are available


In [51]:
df = pd.read_csv("hnh.csv")
df['Images'][0]

'https://handh.blob.core.windows.net/stock/20061543-134-small.jpg?v=63875555356623, https://handh.blob.core.windows.net/stock/20061543-125-small.jpg?v=63874462358873, https://handh.blob.core.windows.net/stock/20061543-36-small.jpg?v=63874461874970, https://handh.blob.core.windows.net/stock/20061543-38-small.jpg?v=63874461876700, https://handh.blob.core.windows.net/stock/20061543-44-small.jpg?v=63874461881830, https://handh.blob.core.windows.net/stock/20061543-45-small.jpg?v=63874461882537, https://handh.blob.core.windows.net/stock/20061543-35-small.jpg?v=63874461873830, https://handh.blob.core.windows.net/stock/20061543-37-small.jpg?v=63874461875890, https://handh.blob.core.windows.net/stock/20061543-39-small.jpg?v=63874461877670, https://handh.blob.core.windows.net/stock/20061543-40-small.jpg?v=63874461878407, https://handh.blob.core.windows.net/stock/20061543-41-small.jpg?v=63874461879223, https://handh.blob.core.windows.net/stock/20061543-42-small.jpg?v=63874461880090, https://handh

### img downloader

In [52]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Title", "Images"]]

def download_images(data, main_folder="Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Title"] 
        image_urls = row["Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_1_20061543-134-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_2_20061543-125-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_3_20061543-36-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_4_20061543-38-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_5_20061543-44-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_6_20061543-45-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_7_20061543-35-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_8_20061543-37-small.jpg
Downloaded: Images\1977 Ferrari 512 BB 'Wide Body'\1977 Ferrari 512 BB 'Wide Body'_9_20061543-39-small.jpg
Downloaded: Images\1977 Ferrari 512

KeyboardInterrupt: 